# 정부 출자 펀드(모태펀드) 운용 방식 및 민간 투자 활성화 방안 논의

**일시**: 미정
**참석자**: 정보 없음

---

## 회의 요약
정부 출자 펀드(모태펀드)의 운용 현황을 점검하고, 민간 투자 유치를 활성화하기 위한 방안으로 정부가 손실을 우선 분담하는 '후순위 출자' 방식 도입을 중점적으로 논의함. 이를 통해 연기금 등 보수적 투자 기관의 참여를 유도하고, 초기 투자 시장의 마중물 역할을 강화하기로 함.

---

## 핵심 논의 내용

### 모태펀드 운용 현황 및 성과
- **펀드 규모**: 올해 정부 출자 펀드 규모는 단일 부처 기준 1.1조 원이며, 타 부처 출자액을 포함하면 총 2.0조 원에 달함.
- **레버리지 효과**: 정부 출자금은 민간 자본과 결합하여 통상 3~4배의 레버리지 효과를 내고 있음.
- **과거 수익률**: 모태펀드는 지난 20년간 연평균 8%의 수익률을 기록했으며, 특히 최근 5년간은 10% 이상의 높은 수익률을 달성하며 민간 펀드에 준하는 우수한 실적을 보임.
- **수익 구조**: 소수 투자 기업의 성공적인 IPO(기업공개) 등 엑싯(Exit)이 전체 펀드 수익률을 견인하는 구조임.

### 민간 투자 활성화를 위한 후순위 출자 방식 도입
- **제안 내용**: 정부가 공공 재원으로 운용되는 펀드의 손실을 먼저 분담하는 '후순위 출자' 방식을 도입할 것을 제안함.
- **기대 효과**: 정부가 10~20%의 초기 손실을 우선 감당할 경우, 민간 투자자(특히 연기금 등)의 원금 손실 위험이 크게 감소하여 더 많은 민간 자금 유치가 가능할 것으로 기대됨.
- **진행 현황**: 현재 퇴직 연금 등의 참여를 유도하기 위해 관계 부처 간 후순위 출자 방식에 대한 협의 및 설계가 진행 중임.
- **참고 사례**: 금융위원회는 이미 성장 기업을 대상으로 후순위 자금을 활용하여 최대 10배의 레버리지를 일으키는 기법을 사용하고 있음.

---

## 액션 아이템
- **모태펀드 후순위 출자 방식 세부 모델 설계**: 관계 부처 협의, 민간 투자(연기금 등) 유치 활성화를 위함, 기한: 미정

---

## 향후 계획
- 설계된 후순위 출자 모델을 바탕으로 연기금, 퇴직 연금 등 민간 기관 투자자들과의 협의를 본격적으로 추진할 계획임.
- 다음 회의에서는 구체적인 펀드 설계안과 민간 투자 유치 전략에 대해 논의할 예정임.

# Minute AI - System Architecture Flowchart

## 📊 Overall System Architecture

```mermaid
graph TB
    subgraph "1. Upload & Processing"
        A[User Upload Audio File] --> B[STT Processing<br/>Whisper API]
        B --> C[Speaker Diarization]
        C --> D[Segment Generation]
    end

    subgraph "2. Storage Layer"
        D --> E1[SQLite DB<br/>개별 대화 저장]
        D --> E2[Smart Chunking<br/>화자/시간 기반]
        E2 --> E3[Gemini 2.5 Flash<br/>메타데이터 제거]
        E3 --> E4[Vector DB ChromaDB<br/>청킹된 문서 저장]
    end

    subgraph "3. Viewer Interface"
        F[Meeting Viewer Page] --> G1[스크립트 탭]
        F --> G2[문단 요약 탭]
        F --> G3[회의록 탭]
    end

    subgraph "4. AI Processing"
        H1[요약 생성<br/>Gemini API] --> I1[Subtopic Summary]
        H2[회의록 생성<br/>Gemini API] --> I2[Formal Minutes]
        I1 --> J1[Vector DB<br/>meeting_subtopic]
        I2 --> J2[SQLite DB<br/>meeting_minutes]
    end

    subgraph "5. Retrieval System"
        K[Search Query] --> L{Retriever Type}
        L -->|similarity| M1[Similarity Search]
        L -->|mmr| M2[MMR Search]
        L -->|self_query| M3[Self Query]
        L -->|similarity_score_threshold| M4[Threshold Filter]
        M1 --> N[Search Results]
        M2 --> N
        M3 --> N
        M4 --> N
    end

    E1 --> F
    E4 --> H2
    E4 --> K
    E1 --> H1
```

## 🔄 Detailed Data Flow

### 1️⃣ Upload & STT Processing Flow

```
Audio File Upload
    ↓
📂 Save to /uploads/
    ↓
🎤 Whisper API Transcription
    ↓
👥 Speaker Diarization
    ↓
📝 Segment Generation
    ├─ speaker_label
    ├─ segment (text)
    ├─ start_time
    └─ end_time
```

### 2️⃣ Smart Chunking Process

```python
# Chunking Logic
for segment in segments:
    current_chunk += segment

    # 청크 분리 조건:
    # 1. 청크 크기 > 1000자
    # 2. 시간 간격 > 60초
    # 3. 화자 변경 + 현재 청크 > 500자

    if should_split:
        formatted_chunk = f"[Speaker {speaker}, {time}] {text}"
        ↓
        cleaned_chunk = gemini_2.5_flash.clean(formatted_chunk)
        ↓
        chunks.append(cleaned_chunk)
```

**Metadata per Chunk:**
- `meeting_id`: 회의 고유 ID
- `title`: 회의 제목
- `meeting_date`: 회의 일시
- `audio_file`: 오디오 파일명
- `chunk_index`: 청크 순서
- `start_time`: 청크 시작 시간
- `end_time`: 청크 종료 시간
- `speaker_count`: 청크 내 화자 수

### 3️⃣ Minutes Generation Flow

```
User clicks "회의록 생성"
    ↓
GET /api/generate_minutes/{meeting_id}
    ↓
📚 Retrieve chunks from Vector DB
    ├─ ORDER BY chunk_index ASC
    ├─ Ensures sequential order
    └─ Full meeting context preserved
    ↓
🤖 Gemini API: generate_minutes(title, transcript, chunks)
    ├─ Input: Chunked documents
    ├─ Process: Structured meeting minutes
    └─ Output: Formal markdown format
    ↓
💾 Save to SQLite meeting_minutes table
    ↓
✅ Display on viewer (HTML conversion)
```

**Key Change:**
- ❌ OLD: Minutes generated from subtopic summaries
- ✅ NEW: Minutes generated directly from chunks

### 4️⃣ Summary Generation Flow

```
User clicks "요약하기"
    ↓
POST /api/summarize/{meeting_id}
    ↓
📝 Retrieve full transcript from SQLite
    ↓
🤖 Gemini API: subtopic_generate(title, transcript)
    ├─ Analyzes entire conversation
    ├─ Identifies key topics
    └─ Generates structured summary
    ↓
💾 Save to Vector DB meeting_subtopic
    ↓
✅ Display on viewer (HTML conversion)
```

### 5️⃣ Enhanced Search System

```python
# 4 Retriever Types

1. Similarity Search
   - Basic cosine similarity
   - Returns top-k most similar chunks
   - Fast and reliable

2. MMR (Maximal Marginal Relevance)
   - fetch_k: 20 (initial candidates)
   - lambda_mult: 0.5 (diversity vs relevance)
   - Reduces redundancy

3. Self Query
   - Natural language filtering
   - Auto-extracts metadata filters
   - Smart query parsing

4. Similarity Score Threshold
   - score_threshold: 0.0-1.0
   - Filters by minimum similarity
   - Quality control
```

## 🗄️ Database Schema

### SQLite Database (`minute_ai.db`)

**Table: `meetings`**
```sql
CREATE TABLE meetings (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    meeting_id TEXT UNIQUE NOT NULL,
    title TEXT NOT NULL,
    meeting_date DATETIME NOT NULL,
    audio_file TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
```

**Table: `segments`**
```sql
CREATE TABLE segments (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    meeting_id TEXT NOT NULL,
    segment TEXT NOT NULL,
    speaker_label TEXT NOT NULL,
    start_time REAL NOT NULL,
    end_time REAL NOT NULL,
    title TEXT,
    meeting_date DATETIME,
    audio_file TEXT,
    FOREIGN KEY (meeting_id) REFERENCES meetings(meeting_id)
);
```

**Table: `meeting_minutes`**
```sql
CREATE TABLE meeting_minutes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    meeting_id TEXT UNIQUE NOT NULL,
    title TEXT NOT NULL,
    meeting_date DATETIME NOT NULL,
    minutes_content TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (meeting_id) REFERENCES meetings(meeting_id)
);
```

### Vector Database (ChromaDB)

**Collection: `meeting_chunk`**
- **Purpose**: RAG-optimized semantic chunks
- **Embedding**: OpenAI text-embedding-3-small
- **Metadata**:
  - `meeting_id`, `title`, `meeting_date`, `audio_file`
  - `chunk_index`, `start_time`, `end_time`, `speaker_count`

**Collection: `meeting_subtopic`**
- **Purpose**: AI-generated subtopic summaries
- **Embedding**: OpenAI text-embedding-3-small
- **Metadata**:
  - `meeting_id`, `title`, `meeting_date`, `audio_file`

## 🛠️ Tech Stack

| Category | Technology | Purpose |
|----------|-----------|---------|
| **Backend** | Flask | Web framework |
| **Database** | SQLite | Structured data storage |
| **Vector DB** | ChromaDB | Semantic search |
| **Embeddings** | OpenAI text-embedding-3-small | Vector representations |
| **STT** | Whisper API | Speech-to-text |
| **AI Processing** | Gemini 2.5 Flash | Text cleaning, summarization, minutes generation |
| **Chunking** | Custom Smart Chunking | Speaker/time-aware splitting |
| **Frontend** | Vanilla JavaScript | Interactive viewer |
| **Audio Player** | HTML5 Audio API | Synchronized playback |

## 🔍 Search API Usage Examples

### Similarity Search
```python
POST /api/search
{
    "query": "프로젝트 일정에 대한 논의",
    "db_type": "chunk",
    "retriever_type": "similarity",
    "k": 5
}
```

### MMR Search (Diverse Results)
```python
POST /api/search
{
    "query": "예산 관련 이슈",
    "db_type": "chunk",
    "retriever_type": "mmr",
    "k": 5,
    "mmr_fetch_k": 20,
    "mmr_lambda_mult": 0.7
}
```

### Threshold Filtering
```python
POST /api/search
{
    "query": "기술 스택 결정",
    "db_type": "chunk",
    "retriever_type": "similarity",
    "score_threshold": 0.75,
    "k": 10
}
```

## ⏱️ Processing Timeline

```
Audio Upload
    ↓ ~1-5분 (파일 크기에 따라)
STT Processing (Whisper API)
    ↓ ~10-30초
Smart Chunking + Gemini Cleaning
    ↓ ~5-15초
Vector DB Storage
    ↓
[Ready for Search & Minutes Generation]
    ↓
Summary Generation (Optional)
    ↓ ~20-60초 (회의 길이에 따라)
Vector DB Storage (meeting_subtopic)
    ↓
Minutes Generation
    ↓ ~30-90초 (청크 수에 따라)
SQLite Storage (meeting_minutes)
```

## 🎯 Key Features

1. **Dual Storage Strategy**
   - SQLite: Fast retrieval, structured queries
   - Vector DB: Semantic search, RAG optimization

2. **Smart Chunking Algorithm**
   - Speaker change detection
   - Time gap analysis (60s threshold)
   - Size-based splitting (1000 char max)
   - Preserves conversational context

3. **Gemini-Powered Text Cleaning**
   - Removes `[Speaker X, MM:SS]` formatting
   - Preserves semantic meaning
   - Optimized for embedding quality

4. **Flexible Search System**
   - 4 retriever types
   - Configurable parameters
   - Metadata filtering support

5. **Progressive Enhancement**
   - Upload → Immediate viewing
   - Optional summary generation
   - On-demand minutes creation
   - No dependencies between features

## 📁 Project Structure

```
minute_ai/
├── app.py                      # Main Flask application
├── database/
│   └── minute_ai.db           # SQLite database
├── utils/
│   ├── stt.py                 # Whisper STT & Gemini processing
│   ├── db_manager.py          # SQLite operations
│   ├── vector_db_manager.py   # ChromaDB operations
│   └── validation.py          # Input validation
├── templates/
│   ├── index.html             # Upload page
│   ├── viewer.html            # Meeting viewer
│   └── notes.html             # Meeting list
├── static/
│   ├── css/
│   └── js/
│       └── viewer.js          # Viewer interactions
└── uploads/                    # Audio file storage
```

## 🚀 Future Enhancements

- [ ] Real-time transcription
- [ ] Multi-language support
- [ ] Custom chunking strategies
- [ ] Advanced RAG techniques (HyDE, CRAG)
- [ ] Export to PDF/DOCX
- [ ] Collaborative editing
- [ ] Meeting analytics dashboard

---

**Last Updated**: 2025-11-03
**Version**: 2.0 (Chunk-based architecture)
